In [ ]:
import cv2 as cv
import numpy as np
import mlflow
import dagshub
import os
import pathlib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.svm import SVC

In [ ]:
PATH = pathlib.Path('C:\\Users\\vinar\\Downloads\\FER2013-augmented')
LABELS = os.listdir(PATH)

In [ ]:
dagshub.init(repo_owner='IdjiotSandwiches', repo_name='face-emotion-recognition', mlflow=True)

In [ ]:
K_SIZE = (31,31)
SIGMA = 4
THETA_RANGE = np.arange(0, np.pi, np.pi/128)
LAMBD = 10.0
GAMMA = 0.5
PSI = 0

gabor_params = {
    'ksize': K_SIZE,
    'sigma': SIGMA,
    'lambd': LAMBD,
    'gamma': GAMMA,
    'psi': PSI
}

In [ ]:
images = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\FER2013\\images.npy')
labels = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\FER2013\\labels.npy')

In [ ]:
TEST_SIZE = 0.2
RANDOM_STATE = 42

split_params = {
    'test_size': TEST_SIZE,
    'random_state': RANDOM_STATE
}

X_train, X_test, y_train, y_test = train_test_split(images, labels, **split_params)

In [ ]:
C = 1.0
KERNEL = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
DEGREE = [3,4]
GAMMA = ['scale', 'auto']

for kernel in KERNEL:
    for degree in DEGREE:
        for gamma in GAMMA:

            model_params = {
                'C': C,
                'kernel': kernel,
                'degree': degree,
                'gamma': gamma
            }

            model = SVC(**model_params)

            predict = model.predict(X_test)

            metrics = {
                'accuracy': accuracy_score(y_test, predict),
                'precision': precision_score(y_test, predict, average='macro'),
                'recall': recall_score(y_test, predict, average='macro'),
                'f1': f1_score(y_test, predict, average='macro'),
                'auc_score': roc_auc_score(y_test, predict_proba, multi_class='ovr', average='macro')
            }

            print(f"Accuracy: {metrics['accuracy']}\nPrecision: {metrics['precision']}\nRecall: {metrics['recall']}\nF1 Score: {metrics['f1']}\nAUC Score: {metrics['auc_score']}")
            report = classification_report(y_test, predict)
            print(report)

            mlflow.set_experiment("SVM_Model FER2013 augmented")
            mlflow.set_tracking_uri("https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow")

            with mlflow.start_run(run_name=f'SVM kernel={kernel} degree={degree} gamma={gamma}'):
                mlflow.log_params(gabor_params)
                mlflow.log_params(calib_params)
                mlflow.log_params(model_params)
                mlflow.log_params(kfold_params)
                mlflow.log_param('floating_point', FLOATING_POINT)
                mlflow.log_param('image_size', IMAGE_SIZE)
                mlflow.log_param('PCA_n_components', N_COMPONENTS)
                mlflow.log_param('gaussian_blur', BLUR)
                mlflow.log_metrics(metrics)
                mlflow.sklearn.log_model(
                    sk_model=calib_model,
                    artifact_path='KNN Model',
                    input_example=X_train[:1]
                )
